# Table on Content
    1. Importing Libraries & data
    2. Security Implications
    3. Regional segmentation
      3.1. Spending Habits across Regions
    4. Customers with low revenue
    5. Merging Departments Dataset
    7. Drop unnecessary columns
    6. Clean Data and Export Dataframes

## 1. Importing Libraries & data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Create path
path = r'/Users/Dulce/12-2023 Instacart Basket Analysis'

In [3]:
# Import ords_prods_custs
ords_prods_custs = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_custs.pkl'))

In [4]:
# Delete unnecessary columns to reduce volume
columns_to_drop = ['aisle_id', 'Gender', '_merge']
ords_prods_custs = ords_prods_custs.drop(columns=columns_to_drop)

In [5]:
# Create a color palette with 10 distinct colors
colors = ['#FFB6C1', '#FFA7B9', '#FF98B2', '#FF89AA', '#FF7AA2', '#FF6B9A', '#FF5C93', '#FF4D8B', '#FF3E83', '#FF2F7B']

## 2. Security Implications

There are some columns which could potentially be traced back to a particular person. We are working with PII in the columns First Name and Last Name so, I will drop them.

In [6]:
# Drop columns First Name and Last Name
ords_prods_custs = ords_prods_custs.drop(columns = ['First Name', 'Last Name'])

## 3. Regional Segmentation

In [7]:
# Create Lists for Regions
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [8]:
# Derive new variable "Region" and placing lists
ords_prods_custs.loc[ords_prods_custs['State'].isin(Northeast), 'region'] = 'Northeast'
ords_prods_custs.loc[ords_prods_custs['State'].isin(Midwest), 'region'] = 'Midwest'
ords_prods_custs.loc[ords_prods_custs['State'].isin(South), 'region'] = 'South'
ords_prods_custs.loc[ords_prods_custs['State'].isin(West), 'region'] = 'West'

### 3.1. Spending Habits across Regions

In [9]:
# Create crosstab
crosstab_normalized = pd.crosstab(ords_prods_custs['spender_type_flag'], ords_prods_custs['region'], dropna=False, normalize='index')

# Multiply the normalized values by 100 to get percentages
crosstab_normalized_percent = (crosstab_normalized * 100).round(0).astype(str) + '%'
crosstab_normalized_percent

region,Midwest,Northeast,South,West
spender_type_flag,,,,
High spender,25.0%,17.0%,33.0%,25.0%
Low spender,23.0%,18.0%,33.0%,26.0%


## 4. Customers with low revenue

In [10]:
# Flag Customers with Low number of orders <5
ords_prods_custs.loc[ords_prods_custs['quantity'] < 5, 'activity_flag'] = 'Low Activity'

In [11]:
# Flag Customers with non-low number of orders >=5
ords_prods_custs.loc[ords_prods_custs['quantity'] >= 5, 'activity_flag'] = 'Non-Low Activity'

In [12]:
# Check results for 'activity_flag'
ords_prods_custs['activity_flag'].value_counts(dropna = False)

activity_flag
Non-Low Activity    24414877
Low Activity         7989982
Name: count, dtype: int64

In [13]:
# Create a subset for 'Low Activity' users
df_low_activity = ords_prods_custs[ords_prods_custs['activity_flag'] == 'Low Activity']
df_low_activity.shape

(7989982, 30)

In [14]:
# Create a subset for 'Non-Low Activity' users for the CFO
df_Nolow_activity = ords_prods_custs[ords_prods_custs['activity_flag'] == 'Non-Low Activity']
df_Nolow_activity.shape

(24414877, 30)

# 5.Merging Departments Dataset

In [15]:
# Import departments
depts = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'))

In [16]:
# Check columns names and rows
depts.head()

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [17]:
# Check Data Type for depts 
depts.dtypes

Unnamed: 0     int64
department    object
dtype: object

In [18]:
# Change column name Unnamed: 0 and check
depts.rename(columns = {'Unnamed: 0' : 'department_id'}, inplace = True)
depts.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [19]:
# Convert department_id into string
depts['department_id'] = depts['department_id'].astype(str)
depts['department_id'].dtype

dtype('O')

In [20]:
# Align data format for department_id in final_df (delete ".0")
ords_prods_custs['department_id'] = ords_prods_custs['department_id'].astype(str).str.replace('.0', '')
ords_prods_custs['department_id'].dtype

dtype('O')

# Delete old '_merge' column
ords_prods_custs = ords_prods_custs.drop(columns = ['_merge'])

In [21]:
# Merge Department.csv with final_df.pkl: Key = department_id
final_df = ords_prods_custs.merge(depts, on = 'department_id', indicator = True)

# 6. Clean Data and Export Dataframes

# Export Non-Low Activity dataframe
df_Nolow_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'Normal Activity.pkl'))

In [22]:
# Export final dataframe
final_df.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'Final_DF.pkl'))